<h3>Importing necessary packages for notebook.</h3>

In [180]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

<h3>Make url call, get response, and parse the response contents using Beautiful Soup.</h3>

In [181]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
response = requests.get(url)
results_page = BeautifulSoup(response.content, "html.parser")

<h3>Extract the table head and table rows from the results.</h3>

In [182]:
table = results_page.find("table")
table_items = table.find_all("tr")
table_head = table_items[0].find_all("th")
table_rows = table_items[1:]

<h3>Create the dataframe with it being initialized as empty with requisite column names coming from table head.</h3>

In [183]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
toronto_neighborhoods = pd.DataFrame(columns = column_names)

<h3>Loop over the rows in table rows and append the applicable data in the dataframe. Ignore data where Borough is not assigned and assign Neighborhood to Borough if Borough is assigned while Neighborhood is not. If more than one neighborhood is assigned to the same postal code include all neighborhoods in same neighborhood column of dataframe for applicable postal code.</h3>

In [184]:
for row in table_rows:
    
    row = row.find_all("td")
    PostalCode = row[0].get_text()
    Borough = row[1].get_text()
    Neighborhood = row[2].get_text().strip("\n")
    
    if Neighborhood == "Not assigned" and Borough != "Not assigned":
        Neighborhood = Borough
    
    if Borough == "Not assigned":
        continue
    else:
        if PostalCode in toronto_neighborhoods["PostalCode"].tolist():
            index = toronto_neighborhoods["PostalCode"].tolist().index(PostalCode)
            toronto_neighborhoods.loc[[index], ["Neighborhood"]] = toronto_neighborhoods.loc[[index], ["Neighborhood"]] + ", " + Neighborhood 
        else:
            toronto_neighborhoods = toronto_neighborhoods.append({
                "PostalCode": PostalCode,
                "Borough": Borough,
                "Neighborhood": Neighborhood
            }, ignore_index = True)

<h3>See the dataframe and its contents.</h3>

In [185]:
toronto_neighborhoods

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


<h3>See what shape the datafame is.</h3>

In [186]:
toronto_neighborhoods.shape

(103, 3)

<h3>Read in a csv file containg latitude and longitude coordinates for the applicable toronto postal codes and store the data in a dataframe. Then join the two daataframes so all data is in one dataframe.</h3>

In [187]:
lat_long_df = pd.read_csv("./Geospatial_Coordinates.csv")
toronto_neigh_with_coord = toronto_neighborhoods.join(lat_long_df.set_index(["Postal Code"]), on = "PostalCode")
toronto_neigh_with_coord

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


<h3>Import Folium to display map and bubbles corresonding to neighborhood locations.</h3>

In [188]:
import folium

<h3>Create and show map of Toronto with bubbles corresponding to the neighborhoods.</h3>

In [189]:
map_Toronto = folium.Map(location=[43.715, -79.3832], zoom_start=10.5)

# add markers to map
for lat, lng, label in zip(toronto_neigh_with_coord['Latitude'], toronto_neigh_with_coord['Longitude'], toronto_neigh_with_coord['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7.5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

<h3>Define some needed inputs for Foursquare queries.</h3>

In [190]:
CLIENT_ID = 'SQNBAVJBDG0POT5KFY2SVAOMWNUZB34ZVD4BR5XNFCNZXO3D' # your Foursquare ID
CLIENT_SECRET = '4KRJRO44XCYV0NLA5F5JROK2N5P3PQDODJEZYYFNCXQRJWRK' # your Foursquare Secret
VERSION = '20190601' # Foursquare API version

LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&time=any&day=any'.format(
    CLIENT_ID, CLIENT_SECRET, 43.657162, -79.378937, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=SQNBAVJBDG0POT5KFY2SVAOMWNUZB34ZVD4BR5XNFCNZXO3D&client_secret=4KRJRO44XCYV0NLA5F5JROK2N5P3PQDODJEZYYFNCXQRJWRK&ll=43.657162,-79.378937&v=20190601&radius=500&limit=100&time=any&day=any'

<h3>Define function that will create and return a dataframe of the results from succesive api calls and responses from Foursqaure api corresponding to all pairs of names, latitudes, and longitudes with venues within radius.</h3>

In [191]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&day=any&time=any'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h3>Filter out neighborhoods that are not in a Borough containg the word Toronto. Done to result in a more concentrated Toronto which neighborhoods will have more venues so as not to have neighborhoods with small observation sizes.</h3>

In [192]:
toronto_boroughs = toronto_neigh_with_coord[toronto_neigh_with_coord['Borough'].str.contains('Toronto')]
toronto_boroughs

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
25,M6G,Downtown Toronto,Christie,43.669542,-79.422564
30,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
31,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
36,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


<h3>Call getNearbyVenues function using the names, latitudes, and longitudes from toronto_boroughs dataframe coressponding to the neighborhoods.</h3>

In [193]:
toronto_venues = getNearbyVenues(names=toronto_boroughs['Neighborhood'],
                                   latitudes=toronto_boroughs['Latitude'],
                                   longitudes=toronto_boroughs['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

<h3>Observe the venues dataframe shape and inspect the first five observations.</h3>

In [194]:
print(toronto_venues.shape)
toronto_venues.head()

(3092, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Harbourfront, Regent Park",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


<h3>Inspect to see how many observations each neighborhood has.</h3>

In [195]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"Brockton, Exhibition Place, Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,48,48,48,48,48,48
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",38,38,38,38,38,38
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100


<h3>Create dataframe using one hot encoding for venues with a one corrsponding to true for that venue corssopnding to that venue category.</h3>

In [196]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h3>Create and inspect the frequency of each venue category in the coressponding neighborhood.</h3>

In [197]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,...,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.00,0.00,0.00,0.00,0.0000,0.0000,0.040000,0.00,0.00,...,0.01,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.010000
1,Berczy Park,0.00,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.00,0.01,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.00,0.00,0.00,0.00,0.0000,0.0000,0.020833,0.00,0.00,...,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.00,0.00,0.00,0.00,0.0625,0.0625,0.000000,0.00,0.00,...,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.000000
5,"Cabbagetown, St. James Town",0.00,0.00,0.00,0.00,0.0000,0.0000,0.026316,0.00,0.00,...,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.000000
6,Central Bay Street,0.00,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.01,0.020000,0.00,0.00,0.000000,0.00,0.010000,0.00,0.0000,0.010000
7,"Chinatown, Grange Park, Kensington Market",0.00,0.00,0.00,0.00,0.0000,0.0000,0.000000,0.00,0.00,...,0.00,0.080000,0.01,0.00,0.030000,0.00,0.010000,0.00,0.0000,0.020000
8,Christie,0.00,0.00,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.020000,0.00,0.00,0.010000,0.00,0.010000,0.00,0.0000,0.000000
9,Church and Wellesley,0.00,0.00,0.01,0.01,0.0000,0.0000,0.010000,0.00,0.00,...,0.00,0.010000,0.01,0.01,0.010000,0.00,0.000000,0.01,0.0100,0.010000


<h3>See each neighborhood and the top five venue categories with their corresponding frequencies for that neighborhood. </h3>

In [198]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0                 Café  0.06
1          Coffee Shop  0.05
2                Hotel  0.05
3  American Restaurant  0.04
4              Theater  0.04


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2                Hotel  0.05
3             Beer Bar  0.04
4  Japanese Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.06
2  Furniture / Home Store  0.04
3                     Bar  0.04
4      Tibetan Restaurant  0.03


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0                Park  0.10
1             Brewery  0.06
2         Coffee Shop  0.06
3  Italian Restaurant  0.04
4         Pizza Place  0.04


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
     

<h3>Function to help sort the top most common venues for a nighborhood.</h3>

In [199]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h3>Create and inspect new dataframe consisting of each nighborhood and its ten most common venues categories.</h3>

In [200]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,American Restaurant,Theater,Pizza Place,Steakhouse,Sushi Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Café,Hotel,Beer Bar,Restaurant,Japanese Restaurant,Park,Bakery,Cocktail Bar,Art Gallery
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,Bar,Furniture / Home Store,Restaurant,Tibetan Restaurant,Bakery,Arts & Crafts Store,Sandwich Place,Gift Shop
3,Business Reply Mail Processing Centre 969 Eastern,Park,Brewery,Coffee Shop,Pizza Place,Burrito Place,Pet Store,Sushi Restaurant,Italian Restaurant,Restaurant,Steakhouse
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Harbor / Marina,Café,Scenic Lookout,Garden,Airport,Airport Lounge,Sculpture Garden,Dog Run,Dance Studio


<h3>Import Kmeans from sklearn to be able to do kmeans clustering.</h3>

In [201]:
from sklearn.cluster import KMeans

<h3>Create and fit kmeans model to the data.</h3>

In [202]:
# set number of clusters
kclusters = 10

toronto_grouped_clustering =toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 9, 4, 2, 1, 3, 9, 1, 1])

<h3>Merge boroughs dataframe with neighborhood venue dataframe containg each neighborhood and its top 10 most common venue categories.</h3>

In [203]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



toronto_merged = toronto_boroughs

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how = 'right')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,5,Coffee Shop,Italian Restaurant,Theater,Café,Restaurant,Bakery,Breakfast Spot,Park,Diner,Mexican Restaurant
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,3,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Gastropub,Clothing Store,Tea Room,Restaurant,Ramen Restaurant,Italian Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,3,Café,Coffee Shop,Restaurant,Hotel,Bakery,Italian Restaurant,Cosmetics Shop,Gastropub,Breakfast Spot,American Restaurant
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,4,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Japanese Restaurant,Bar,Park,Beach,Sandwich Place,Bakery
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,3,Coffee Shop,Café,Hotel,Beer Bar,Restaurant,Japanese Restaurant,Park,Bakery,Cocktail Bar,Art Gallery


<h3>Import necessary matplotlib components before using in mapping.</h3>

In [204]:
import matplotlib.cm as cm
import matplotlib.colors as colors

<h3>Create and show map consisting of the neighborhoods in the merged dataframe with each bubble showing the location of the neighborhood and the color of bubble corresponding to the cluster it is assigned with neighborhoods assigned to the same cluster having the same color.</h3>

In [205]:
map_clusters = folium.Map(location=[43.675, -79.3832], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>From above map one can observe that in general neighborhoods that are a similar distance from the center of downtown toronto are are clustered together. Since the criteria for clustering was similarity in venues for the neighborhoods this implies that similar venues are within neighborhoods of similar distance from center of Toronto. This then also might imply that certain types of customers either live or frequent venues in those locations creating the demand for the same type of venues in those neighborhoods.</h3>

<h3>Below is each cluster one after another where you can look and inspect the neighborhoods in the cluster and compare the top 10 most common venue categories amongst the neighborhoods.</h3>

In [206]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,West Toronto,0,Café,Coffee Shop,Park,Italian Restaurant,Bar,Bakery,Grocery Store,Pharmacy,Portuguese Restaurant,Brewery
68,Central Toronto,0,Park,Café,Italian Restaurant,Coffee Shop,Bank,Burger Joint,Trail,Sushi Restaurant,Japanese Restaurant,Mediterranean Restaurant


In [207]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,1,Korean Restaurant,Café,Grocery Store,Coffee Shop,Ice Cream Shop,Cocktail Bar,Indian Restaurant,Japanese Restaurant,Sandwich Place,Bar
47,East Toronto,1,Indian Restaurant,Coffee Shop,Beach,Café,Burrito Place,Bakery,Sandwich Place,Butcher,Spa,Restaurant
74,Central Toronto,1,Café,Vegetarian / Vegan Restaurant,Italian Restaurant,Restaurant,Coffee Shop,Museum,Pizza Place,Bakery,Pub,Grocery Store
96,Downtown Toronto,1,Park,Gastropub,Café,Diner,Coffee Shop,Japanese Restaurant,Steakhouse,Deli / Bodega,Taiwanese Restaurant,Pub
99,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Café,Grocery Store,Park,Gym,Italian Restaurant,Gay Bar


In [208]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
87,Downtown Toronto,2,Coffee Shop,Harbor / Marina,Café,Scenic Lookout,Garden,Airport,Airport Lounge,Sculpture Garden,Dog Run,Dance Studio


In [209]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Downtown Toronto,3,Coffee Shop,Cosmetics Shop,Café,Middle Eastern Restaurant,Gastropub,Clothing Store,Tea Room,Restaurant,Ramen Restaurant,Italian Restaurant
15,Downtown Toronto,3,Café,Coffee Shop,Restaurant,Hotel,Bakery,Italian Restaurant,Cosmetics Shop,Gastropub,Breakfast Spot,American Restaurant
20,Downtown Toronto,3,Coffee Shop,Café,Hotel,Beer Bar,Restaurant,Japanese Restaurant,Park,Bakery,Cocktail Bar,Art Gallery
24,Downtown Toronto,3,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Ramen Restaurant,Arts & Crafts Store,Park,Bubble Tea Shop,Clothing Store,Middle Eastern Restaurant
30,Downtown Toronto,3,Café,Coffee Shop,Hotel,American Restaurant,Theater,Pizza Place,Steakhouse,Sushi Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant
36,Downtown Toronto,3,Coffee Shop,Café,Hotel,Italian Restaurant,Brewery,Restaurant,Scenic Lookout,Park,Aquarium,Bakery
42,Downtown Toronto,3,Café,Hotel,Coffee Shop,Steakhouse,Italian Restaurant,American Restaurant,Concert Hall,Monument / Landmark,Lounge,Gastropub
48,Downtown Toronto,3,Coffee Shop,Hotel,Café,Steakhouse,Concert Hall,American Restaurant,Restaurant,Japanese Restaurant,Beer Bar,Theater
92,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Gastropub,Seafood Restaurant,Beer Bar,Bakery,Art Gallery
97,Downtown Toronto,3,Café,Coffee Shop,Hotel,Steakhouse,Italian Restaurant,American Restaurant,Theater,Gym,Beer Bar,Concert Hall


In [210]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,4,Pub,Coffee Shop,Pizza Place,Breakfast Spot,Japanese Restaurant,Bar,Park,Beach,Sandwich Place,Bakery
54,East Toronto,4,Coffee Shop,Café,Bar,American Restaurant,Bakery,Vietnamese Restaurant,Italian Restaurant,Diner,Sandwich Place,Brewery
69,West Toronto,4,Bar,Café,Coffee Shop,Thai Restaurant,Convenience Store,Italian Restaurant,Sushi Restaurant,Mexican Restaurant,Sandwich Place,Pizza Place
75,West Toronto,4,Bar,Café,Coffee Shop,Pizza Place,Sushi Restaurant,Breakfast Spot,Bakery,Thai Restaurant,Eastern European Restaurant,Park
83,Central Toronto,4,Coffee Shop,Park,Italian Restaurant,Grocery Store,Gym,Pub,Bagel Shop,Sandwich Place,Café,Restaurant
100,East Toronto,4,Park,Brewery,Coffee Shop,Pizza Place,Burrito Place,Pet Store,Sushi Restaurant,Italian Restaurant,Restaurant,Steakhouse


In [211]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,5,Coffee Shop,Italian Restaurant,Theater,Café,Restaurant,Bakery,Breakfast Spot,Park,Diner,Mexican Restaurant
41,East Toronto,5,Greek Restaurant,Coffee Shop,Pub,Café,Fast Food Restaurant,Italian Restaurant,Ice Cream Shop,Pizza Place,Bakery,Furniture / Home Store
67,Central Toronto,5,Coffee Shop,Fast Food Restaurant,Italian Restaurant,Pizza Place,Café,Pharmacy,Gym,Dessert Shop,Sushi Restaurant,Sporting Goods Shop
73,Central Toronto,5,Sporting Goods Shop,Italian Restaurant,Coffee Shop,Park,Café,Mexican Restaurant,Diner,Pharmacy,Spa,Skating Rink
79,Central Toronto,5,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pizza Place,Restaurant,Pub,Gym,Fast Food Restaurant,Indian Restaurant
81,West Toronto,5,Coffee Shop,Café,Bakery,Pizza Place,Italian Restaurant,Falafel Restaurant,Sushi Restaurant,Restaurant,Diner,Gastropub
86,Central Toronto,5,Coffee Shop,Park,Italian Restaurant,Sushi Restaurant,Pizza Place,Pub,Spa,Bagel Shop,Café,Liquor Store


In [212]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
61,Central Toronto,6,Bookstore,Park,College Quad,College Gym,Coffee Shop,Gym / Fitness Center,Café,Trail,Dog Run,Doner Restaurant


In [213]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 7, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Downtown Toronto,7,Coffee Shop,Grocery Store,Bank,Park,Breakfast Spot,Filipino Restaurant,Candy Store,Japanese Restaurant,Metro Station,BBQ Joint


In [214]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 8, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,8,Sushi Restaurant,Asian Restaurant,Pharmacy,Bank,Café,Italian Restaurant,Coffee Shop,Bagel Shop,Japanese Restaurant,Deli / Bodega


In [215]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 9, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,West Toronto,9,Bar,Café,Bakery,Pizza Place,Restaurant,Coffee Shop,Cocktail Bar,Italian Restaurant,Vegetarian / Vegan Restaurant,Asian Restaurant
43,West Toronto,9,Café,Coffee Shop,Bar,Furniture / Home Store,Restaurant,Tibetan Restaurant,Bakery,Arts & Crafts Store,Sandwich Place,Gift Shop
80,Downtown Toronto,9,Café,Bakery,Vegetarian / Vegan Restaurant,Bar,Restaurant,Bookstore,Coffee Shop,Pizza Place,Mexican Restaurant,Arts & Crafts Store
84,Downtown Toronto,9,Café,Vegetarian / Vegan Restaurant,Bar,Mexican Restaurant,Bakery,Vietnamese Restaurant,Coffee Shop,Gaming Cafe,Dumpling Restaurant,Dessert Shop
